<a href="https://colab.research.google.com/github/Tclack88/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module1-rnn-and-lstm/LS_DS_431_RNN_and_LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [0]:
import numpy as np

### import and clean text

In [5]:
!wget https://www.gutenberg.org/files/100/100-0.txt && ls

--2019-12-16 22:41:41--  https://www.gutenberg.org/files/100/100-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5777367 (5.5M) [text/plain]
Saving to: ‘100-0.txt’

100-0.txt           100%[===================>]   5.51M  4.20MB/s    in 1.3s    

2019-12-16 22:41:43 (4.20 MB/s) - ‘100-0.txt’ saved [5777367/5777367]

100-0.txt  sample_data


In [0]:
import re

In [0]:
data = '100-0.txt'
text = ''
with open(data, 'r') as f:
  text += f.read()

text = re.sub(r'[^A-Za-z 0-9]', '',text.replace('\n', ' ').replace('\t', ' ')) # replace newlines and tabs with spaces
text = ' '.join(text.split()) # remove excess spaces between characters

In [8]:
text[:200]

'Project Gutenbergs The Complete Works of William Shakespeare by William Shakespeare This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and w'

### Create relevant variables

In [0]:
chars = list(set(list(text)))

int_char = {i:c for i,c in enumerate(chars)}
char_int = {c:i for i,c in enumerate(chars)}

### Create sequence data 

In [0]:
max_len = 60
step = 5

encoded = [char_int[c] for c in text]

sequences = []
next_chars = []

for i in range(0, len(encoded) - max_len, step):
  sequences.append(encoded[i : i + max_len])
  next_chars.append(encoded[i + max_len])

### create matrix inputs and predicted values (X and y)

In [0]:
X = np.zeros((len(sequences), max_len, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
  for s, char in enumerate(sequence):
    X[i, s, char] = 1
  y[i, next_chars[i]] = 1

## Create LSTM model

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import LambdaCallback

In [0]:
model = Sequential()
model.add(LSTM(128, input_shape=(max_len, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=.01))

### running definitions taken from lecture

In [0]:
import sys
import random

In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - max_len - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + max_len]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, max_len, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_int[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = int_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [34]:
model.fit(X, y, batch_size=512, epochs=5, callbacks=[print_callback])

Train on 1001479 samples
Epoch 1/5
1001472/1001479 [============================>.] - ETA: 0s - loss: 2.0563
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "If the cook help to make the gluttony you help to make the d"
If the cook help to make the gluttony you help to make the death the mean the state and so so shall be the send the common the street the common the country the see the street the more the stream the sea and the son the set the street to the see the seem and so she shall be so me to the send the such a good constant the see the consent the death the send the fair there is so see the s

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


eem and the street and the say the fair present the sender to the country
----- diversity: 0.5
----- Generating with seed: "If the cook help to make the gluttony you help to make the d"
If the cook help to make the gluttony you help to make the deep lay me the constart to death me with the love a the such for he he will comes I must thou art the stand the streams on the sun and seem where be not the sense but were a death in the liver be the body I shall be sir and like thee traiting were the seemd and a house the scance like her heart he so the death of my fier be come to the with the more to he doth he is the proclaim And so will the si
----- diversity: 1.0
----- Generating with seed: "If the cook help to make the gluttony you help to make the d"
If the cook help to make the gluttony you help to make the death MACBETH What nurse boy bele ye follows to if vick certain to me complates procasos Where be but curcing the drunking I shall not be and holding this weasin wings those sequeed 

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN